In [3]:
import numpy as np
import pandas as pd
from textwrap import wrap
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
import sys
from hsbmpy import plot_topic_size, get_max_available_L

In [2]:
directory="/home/fvalle/phd/datasets/breast_HDE/"

In [4]:
L = get_max_available_L(directory)

# topic size

In [ ]:
for l in range(0,L+1):
    plot_topic_size(directory,l)

## Topic O

In [ ]:
df_mv=pd.read_csv("%s/meanVariances.csv"%directory, index_col=[0])
df_o = pd.read_csv("%s/O.dat"%directory, header=None)
df_mv.insert(3, 'occurrence', df_o.values)
#df_mv['occurrence'] = df_mv['occurrence']
df_mv.head()

In [ ]:
for l in range(0,L+1):
    fig = plt.figure(figsize=(15,8))
    ax = fig.subplots(1,2)
    candles = get_candles(directory,l,df_mv,ax[0])
    candlestick2_ohlc(ax[0], candles['open'],candles['high'],candles['low'],candles['close'],width=0.6,colordown='b')
    ax[1].hist((np.array(candles['open'])+np.array(candles['close']))/2, weights=candles['size'], range=(-0.05,1.05), bins=10, histtype='step')
    ax[1].set_xlabel("$O_i", fontsize=18)
    plt.show()
    fig.savefig("%s/topic_Ocandles_level_%d.pdf"%(directory,l))

# Geneontology

In [5]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [6]:
l=L
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,l))

In [7]:
#import gseapy as gs
#gs.get_library_name()

In [8]:
back_sea=[]
for topic in df_topics.columns:
    for g in df_topics[topic].dropna().values:
        try:
            back_sea.append(get_symbol(g))
        except:
            print("error %s"%g)

In [10]:
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/gsea/gsea_level_%d_topic%d.csv"%(directory,l,itopic), index_col=[0])
        print(topic)
    except:
        try:
            symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
            print(topic)
            enriched_topic = get_ontology_df(symbols, background=back_sea).sort_values(by=['Adjusted P-value'], ascending=True)
            enriched_topic = enriched_topic.loc[enriched_topic.index.values[:20],:]
            enriched_topic.to_csv("%s/gsea/gsea_level_%d_topic%d.csv"%(directory,l,itopic))
        except:
            print(*sys.exc_info())
    print(enriched_topic)

Topic 1


2019-12-04 11:32:35,958 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.05
2019-12-04 11:32:42,797 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.05


<class 'FileNotFoundError'> [Errno 2] No such file or directory: '/home/fvalle/phd/datasets/breast_HDE//gsea/gsea_level_4_topic0.csv' <traceback object at 0x7fe671511c48>
                                                   Term  Adjusted P-value  \
4097    GTEX-R55F-1726-SM-2TF4R_breast_male_50-59_years      6.793352e-81   
7011     GTEX-XBEC-1326-SM-4AT69_heart_male_50-59_years      2.143403e-65   
7012     GTEX-OIZG-1126-SM-2HMIU_heart_male_50-59_years      3.809245e-57   
7013   GTEX-WRHU-1226-SM-4E3IJ_heart_female_50-59_years      4.368171e-55   
4098      GTEX-OIZI-0008-SM-2XCFD_skin_male_40-49_years      3.100015e-53   
4099  GTEX-VJWN-0726-SM-3GIJ8_breast_female_50-59_years      5.212389e-53   
4100    GTEX-T6MN-0726-SM-32PML_breast_male_50-59_years      2.349504e-48   
7015     GTEX-XQ3S-0006-SM-4BOQ4_blood_male_20-29_years      8.396112e-48   
7014   GTEX-X8HC-1626-SM-46MWE_heart_female_50-59_years      9.677877e-48   
7016   GTEX-WWYW-1326-SM-3NB2S_heart_female_50-59_years    

2019-12-04 11:33:06,979 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.05
2019-12-04 11:33:09,587 Warning: No enrich terms using library GO_Biological_Process_2018 when cutoff = 0.05
2019-12-04 11:33:12,074 Warning: No enrich terms using library GO_Cellular_Component_2018 when cutoff = 0.05
2019-12-04 11:33:14,566 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.05
2019-12-04 11:33:20,327 Warning: No enrich terms using library GTEx_Tissue_Sample_Gene_Expression_Profiles_down when cutoff = 0.05
2019-12-04 11:33:22,805 Warning: No enrich terms using library Tissue_Protein_Expression_from_Human_Proteome_Map when cutoff = 0.05
2019-12-04 11:33:25,313 Warning: No enrich terms using library KEGG_2019_Human when cutoff = 0.05
2019-12-04 11:33:27,827 Warning: No enrich terms using library NCI-60_Cancer_Cell_Lines when cutoff = 0.05


<class 'FileNotFoundError'> [Errno 2] No such file or directory: '/home/fvalle/phd/datasets/breast_HDE//gsea/gsea_level_4_topic1.csv' <traceback object at 0x7fe671493148>
                                                  Term  Adjusted P-value  \
283  GTEX-NPJ7-0011-R1a-SM-3GACT_brain_female_60-69...          0.004586   
282   GTEX-NPJ7-2726-SM-2I3FT_brain_female_60-69_years          0.005291   
284  GTEX-TSE9-0011-R1A-SM-3DB7E_brain_female_60-69...          0.005555   
285  GTEX-Q2AG-0011-R1A-SM-2HMJI_brain_female_40-49...          0.006725   
287  GTEX-WWYW-0011-R1A-SM-3TW8G_brain_female_50-59...          0.026468   
286  GTEX-T5JC-0011-R1A-SM-32PM6_brain_male_20-29_y...          0.031751   
288  GTEX-WHSE-0011-R3A-SM-3P5ZM_brain_male_20-29_y...          0.037759   
290  GTEX-RVPV-0011-R3A-SM-2TF63_brain_male_60-69_y...          0.040965   
289  GTEX-N7MS-0011-R1a-SM-2HMJG_brain_male_60-69_y...          0.043940   
293  GTEX-RVPU-0011-R1A-SM-2XCAI_brain_male_50-59_y...          0.047

2019-12-04 11:33:31,130 Warning: No enrich terms using library GO_Molecular_Function_2018 when cutoff = 0.05
2019-12-04 11:33:39,670 Warning: No enrich terms using library Human_Phenotype_Ontology when cutoff = 0.05
2019-12-04 11:33:54,491 Warning: No enrich terms using library NCI-60_Cancer_Cell_Lines when cutoff = 0.05


<class 'FileNotFoundError'> [Errno 2] No such file or directory: '/home/fvalle/phd/datasets/breast_HDE//gsea/gsea_level_4_topic2.csv' <traceback object at 0x7fe670fb2688>
                                                   Term  Adjusted P-value  \
2076  GTEX-XUJ4-1926-SM-4BOOV_breast_female_60-69_years      1.873691e-29   
2077  GTEX-UJHI-1426-SM-3DB9C_breast_female_50-59_years      2.571868e-28   
2079  GTEX-WXYG-2226-SM-4E3IM_breast_female_50-59_years      1.100463e-24   
2078  GTEX-SE5C-2126-SM-4BRUJ_breast_female_40-49_years      1.207172e-24   
2080  GTEX-XUW1-2326-SM-4BOO5_breast_female_50-59_years      8.846957e-23   
2082  GTEX-T5JW-2026-SM-4DM63_breast_female_20-29_years      1.367050e-22   
2081  GTEX-T6MO-0326-SM-32QOK_breast_female_40-49_years      1.567748e-22   
2083  GTEX-S4UY-0726-SM-4AD6X_breast_female_40-49_years      9.778274e-22   
2084  GTEX-WRHU-0326-SM-3MJFY_breast_female_50-59_years      1.965659e-21   
2085  GTEX-R53T-1526-SM-48FEK_breast_female_50-59_years    

In [ ]:
topic_pvalues = []
topic_gos = []
for itopic,topic in enumerate(df_topics.columns):
    try:
        enriched_topic = pd.read_csv("%s/gsea/gsea_level_%d_topic%d.csv"%(directory,l,itopic))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
            for goc in enriched_topic['Gene_set'][:10].unique():
                topic_gos.append(goc)
        print(topic)
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure()
x = np.arange(1,1+len(topic_pvalues))
c, _, _ = plt.hist(topic_pvalues, histtype='step', lw=2)
plt.plot([-np.log10(0.05) for _ in np.linspace(1,10,num=10)],np.arange(0,np.max(c)+5,(np.max(c)+5)/10), ls='--', lw=5, label="$\\alpha=0.05$")
plt.xlabel('-log(P-value)', fontsize=16)
plt.ylabel("number of topics")
#plt.ylim(0,0.055)
#plt.yscale('log')
plt.legend(fontsize=16)
fig.savefig("%s/pvaluescrosstopic(%d).png"%(directory,l))

In [ ]:
fig = plt.figure(figsize=(20,10))
gos, goscounts = np.unique(topic_gos,return_counts=True)
plt.barh(["\n".join(wrap(str(l).replace('_',' '),20)) for l in gos], goscounts)
plt.yticks(fontsize=15)
plt.show()
fig.savefig("%s/pvaluecategories(%d).pdf"%(directory,l))